<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open('css/custom.css', 'r').read()
    return HTML(styles)
css_styling()

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2017 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 19.</h5>

<h1 class='lesson_title'>Solving the points-after-touchdown problem</h1>

## Before we begin &mdash; upgrading and importing `stochasticdp`

* I've made some improvements to the internals of `stochasticdp`.

* To upgrade, open a WinPython Command Prompt and type:

```
pip install --upgrade stochasticdp
```

* Now we can import `StochasticDP` from `stochasticdp`:

In [ ]:
from stochasticdp import StochasticDP

## Setting up the data

* In Lesson 18, we worked with the following data:

\begin{alignat*}{2}
  T & = \text{total number of possessions} \\[1ex]
  p_n & = \Pr\{ \text{1-pt. conv. successful for Team $n$} \,|\, \text{1-pt. conv. attempted by Team $n$} \} & \quad & \text{for } n = \text{A}, \text{B} \\
  q_n & = \Pr\{ \text{2-pt. conv. successful for Team $n$} \,|\, \text{2-pt. conv. attempted by Team $n$} \} & \quad & \text{for } n = \text{A}, \text{B} \\[1ex]
  b_1 & = \Pr \{ \text{1-pt. conv. attempted by Team B} \}\\
  b_2 & = \Pr \{ \text{2-pt. conv. attempted by Team B} \}\\[1ex]
  t_n & = \Pr \{ \text{TD by Team $n$ in 1 possession} \}                                                                     & \quad & \text{for } n = \text{A}, \text{B} \\
  g_n & = \Pr \{ \text{FG by Team $n$ in 1 possession} \}                                                                     & \quad & \text{for } n = \text{A}, \text{B} \\
  z_n & = \Pr \{ \text{no score by Team $n$ in 1 possession} \}                                                               & \quad & \text{for } n = \text{A}, \text{B} \\[1ex]
  r & = \Pr \{ \text{Team A wins in overtime} \}
\end{alignat*}

* Let's begin by defining numerical values for this data.

* We can find most of these values from [Pro Football Reference](http://www.pro-football-reference.com/years/2014/).

* For now, let's assume that Team A and Team B are both average 2014 NFL teams.
    - Recall that in 2014, 1-pt. conversions started at the 2-yard line.

* Also, let's assume that Team A wins in overtime with probability 0.5.

In [ ]:
# Total number of possessions
# Drive Averages: 2 * (#Dr) / (G * (# of teams))
T = 23

# 1-pt. conversion success probabilities
# Kicking and Punting: XP%
pA = 0.993
pB = 0.993

# 2-pt. conversion success probabilities
# Scoring Offense: 2PM / 2PA
qA = 0.483
qB = 0.483

# 1-pt. vs 2-pt attempts
# 1-pt.: Scoring Offense: XPA / (XPA + 2PA)
b1 = 0.954
b2 = 1 - b1

# Possession outcome probabilities - Team A
# TD: (Scoring Offense: ATD) / (Drive Averages: #Dr)
# FG: (Scoring Offense: FGM) / (Drive Averages: #Dr)
tA = 0.218
gA = 0.172
zA = 1 - tA - gA

# Possession outcome probabilities - Team B
tB = 0.218
gB = 0.172
zB = 1 - tB - gB

# Probability that Team A wins in OT
r = 0.5

## Initializing the stochastic dynamic program

* Stages:

\begin{align*}
  t = 0, 1, \dots, T - 1 \quad & \leftrightarrow \quad \text{end of possession $t$} \\
  t = T \quad                  & \leftrightarrow \quad \text{end of game}
\end{align*}

In [ ]:
# Number of stages
number_of_stages = T + 1

* States:

\begin{alignat*}{2}
  (n, k, d) \quad \leftrightarrow \quad & \text{Team $n$'s possession just ended} & \quad & \text{for } n \in \{\text{A}, \text{B}\}      \\
                                        & \text{Team $n$ just scored $k$ points}  & \quad & \text{for } k \in \{0, 3, 6\}                 \\
                                        & \text{Team A is ahead by $d$ points}    & \quad & \text{for } d \in \{\dots, -1, 0, 1, \dots,\}
\end{alignat*}

* In Lesson 18, we did not assume a lower or upper bound on $d$, the values that represent Team A's lead. 

* Since we need to have a finite number of states, let's assume $-20 \le d \le 20$.

* Some Python reminders:
    * We can construct a list by 
        - first creating an empty list,
        - and then adding items to it using `.append()`. 
    * For example:
    ```python
    my_list = []
    for i in range(10):
        my_list.append(i)
    ```
    * `range(a)` iterates over the integers `0, 1, ..., a - 1`, while `range(a, b)` iterates over the integers `a, a + 1, ..., b - 1`.

In [ ]:
# Maximum lead for Team A
max_d = 20

# List of states
states = []
for n in ['A', 'B']:
    for k in [0, 3, 6]:
        for d in range(-max_d, max_d + 1):
            states.append((n, k, d))

* Allowable decisions $x_t$ at stage $t$ and state $(n, k, d)$:

\begin{alignat*}{2}
  x_t & \in \{ 1, 2 \} & \quad & \text{if } n = \text{A} \text{ and } k = 6             \\
  x_t & = \text{none}  & \quad & \text{if } n = \text{A} \text{ and } k \in \{0, 3\}    \\
  x_t & = \text{none}  & \quad & \text{if } n = \text{B} \text{ and } k \in \{0, 3, 6\}
\end{alignat*}

In [ ]:
# List of decisions
decisions = [1, 2, 'none']

* Now we can initialize a stochastic DP object called `dp` as follows:

In [ ]:
# Initialize stochastic dynamic program - we want to maximize, so minimize = False
dp = StochasticDP(number_of_stages, states, decisions, minimize=False)

## Transition probabilities from stages $t = 0, 1, \dots, T-2$

* First, let's tackle transitions from the state $(A, 6, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

![State $(A, 6, d)$](img/a6d.png)

* In Lesson 18, we assumed that $d$ could take on an infinite number values.

* On the other hand, here, we have limited $d$ to be between $-20$ and $20$.

* How does this change our transition probabilities?

* For example, suppose $d = -17$ in the diagram above. Then we can model the transition probabilities like this:

![State $(A, 6, -17)$ under decision $x_t = 1$](img/a6d-2.png)

* In other words, if $d$ is supposed to be less than $-20$, then we simply assume that it is the same as having $d = -20$.

* We can do the same thing when $d$ is supposed to be greater than $20$.

* To implement this easily, we can define the transition probabilities like in the cell below.

* _Notes._ 
    - In Python,
    ```python
    a += 3
    ```
    is the same as
    ```python
    a = a + 3
    ```
    - Remember that the transition probabilities and contributions are all initialized to 0.

In [ ]:
# Transition probabilities from (A, 6, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        # 1-point conversion
        dp.transition[('B', 6, max(d - 6, -max_d)), ('A', 6, d), t, 1] += (1 - pA) * tB
        dp.transition[('B', 6, max(d - 5, -max_d)), ('A', 6, d), t, 1] += pA * tB
        dp.transition[('B', 3, max(d - 3, -max_d)), ('A', 6, d), t, 1] += (1 - pA) * gB
        dp.transition[('B', 3, max(d - 2, -max_d)), ('A', 6, d), t, 1] += pA * gB
        dp.transition[('B', 0, d),                  ('A', 6, d), t, 1] += (1 - pA) * zB
        dp.transition[('B', 0, min(d + 1, max_d)),  ('A', 6, d), t, 1] += pA * zB
        
        # 2-point conversion
        dp.transition[('B', 6, max(d - 6, -max_d)), ('A', 6, d), t, 2] += (1 - qA) * tB
        dp.transition[('B', 6, max(d - 4, -max_d)), ('A', 6, d), t, 2] += qA * tB
        dp.transition[('B', 3, max(d - 3, -max_d)), ('A', 6, d), t, 2] += (1 - qA) * gB
        dp.transition[('B', 3, max(d - 1, -max_d)), ('A', 6, d), t, 2] += qA * gB
        dp.transition[('B', 0, d),                  ('A', 6, d), t, 2] += (1 - qA) * zB
        dp.transition[('B', 0, min(d + 2, max_d)),  ('A', 6, d), t, 2] += qA * zB

* In a similar fashion, we can define the remaining transition probabilities.

* From states $(A, 3, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

![State $(A, 3, d)$](img/a3d.png)

In [ ]:
# Transition probabilities from (A, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('B', 6, max(d - 6, -max_d)), ('A', 3, d), t, 'none'] += tB
        dp.transition[('B', 3, max(d - 3, -max_d)), ('A', 3, d), t, 'none'] += gB
        dp.transition[('B', 0, d),                  ('A', 3, d), t, 'none'] += zB

* From states $(A, 0, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

![State $(A, 0, d)$](img/a0d.png)

In [ ]:
# Transition probabilities from (A, 0, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('B', 6, max(d - 6, -max_d)), ('A', 0, d), t, 'none'] += tB
        dp.transition[('B', 3, max(d - 3, -max_d)), ('A', 0, d), t, 'none'] += gB
        dp.transition[('B', 0, d),                  ('A', 0, d), t, 'none'] += zB

* From states $(B, 6, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

![State $(B, 6, d)$](img/b6d.png)

In [ ]:
# Transition probabilities from (B, 6, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('A', 6, min(d + 6, max_d)), ('B', 6, d), t, 'none'] += (1 - b1*pB - b2*qB) * tA
        dp.transition[('A', 3, min(d + 3, max_d)), ('B', 6, d), t, 'none'] += (1 - b1*pB - b2*qB) * gA
        dp.transition[('A', 0, d),                 ('B', 6, d), t, 'none'] += (1 - b1*pB - b2*qB) * zA
        
        dp.transition[('A', 6, min(d + 5, max_d)),  ('B', 6, d), t, 'none'] += b1 * pB * tA
        dp.transition[('A', 3, min(d + 2, max_d)),  ('B', 6, d), t, 'none'] += b1 * pB * gA
        dp.transition[('A', 0, max(d - 1, -max_d)), ('B', 6, d), t, 'none'] += b1 * pB * zA
        
        dp.transition[('A', 6, min(d + 4, max_d)),  ('B', 6, d), t, 'none'] += b2 * qB * tA
        dp.transition[('A', 3, min(d + 1, max_d)),  ('B', 6, d), t, 'none'] += b2 * qB * gA
        dp.transition[('A', 0, max(d - 2, -max_d)), ('B', 6, d), t, 'none'] += b2 * qB * zA

* From states $(B, 3, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

![State $(B, 3, d)$](img/b3d.png)

In [ ]:
# Transition probabilities from (B, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('A', 6, min(d + 6, max_d)), ('B', 3, d), t, 'none'] += tA
        dp.transition[('A', 3, min(d + 3, max_d)), ('B', 3, d), t, 'none'] += gA
        dp.transition[('A', 0, d),                 ('B', 3, d), t, 'none'] += zA

* From states $(B, 0, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

![State $(B, 0, d)$](img/b0d.png)

In [ ]:
# Transition probabilities from (B, 0, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('A', 6, min(d + 6, max_d)), ('B', 0, d), t, 'none'] += tA
        dp.transition[('A', 3, min(d + 3, max_d)), ('B', 0, d), t, 'none'] += gA
        dp.transition[('A', 0, d),                 ('B', 0, d), t, 'none'] += zA

## Transition probabilities from stage $T - 1$

* Now, we can tackle the transitions from stage $T-1$.

* From states $(A, 6, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

![](img/last_a6d.png)

In [ ]:
# Transition probabilities from (A, 6, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    # 1-point conversion
    dp.transition[('A', 6, min(d + 1, max_d)),  ('A', 6, d), T - 1, 1] += pA
    dp.transition[('A', 6, d),                  ('A', 6, d), T - 1, 1] += 1 - pA

    # 2-point conversion
    dp.transition[('B', 0, min(d + 2, max_d)),  ('A', 6, d), T - 1, 2] += qA
    dp.transition[('B', 0, d),                  ('A', 6, d), T - 1, 2] += (1 - qA)

* From states $(A, 3, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

![](img/last_a3d.png)

In [ ]:
# Transition probabilities from (A, 3, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('A', 3, d), ('A', 3, d), T - 1, 'none'] += 1

* From states $(A, 0, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

![](img/last_a0d.png)

In [ ]:
# Transition probabilities from (A, 0, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('A', 0, d), ('A', 0, d), T - 1, 'none'] += 1

* From states $(B, 6, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

![](img/last_b6d.png)

In [ ]:
# Transition probabilities from (B, 6, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('B', 6, max(d - 2, -max_d)),  ('B', 6, d), T - 1, 'none'] += qB * b2
    dp.transition[('B', 6, max(d - 1, -max_d)),  ('B', 6, d), T - 1, 'none'] += pB * b1
    dp.transition[('B', 6, d),                  ('B', 6, d), T - 1, 'none'] += 1 - pB * b1 - qB * b2

* From states $(B, 3, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

![](img/last_b3d.png)

In [ ]:
# Transition probabilities from (B, 3, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('B', 3, d), ('B', 3, d), T - 1, 'none'] += 1

* From states $(B, 0, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

![](img/last_b0d.png)

In [ ]:
# Transition probabilities from (B, 0, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('B', 0, d), ('B', 0, d), T - 1, 'none'] += 1

## Boundary conditions

* Finally, the boundary conditions:

$$
f_T(n, k, d) = \begin{cases}
1 & \text{if } d > 0\\
r & \text{if } d = 0\\
0 & \text{if } d < 0
\end{cases}
\qquad \text{for } n \in \{\text{A}, \text{B}\}, k \in \{0, 3, 6\}, d = -20,\dots,20
$$

In [ ]:
# Boundary conditions
for n in ['A', 'B']:
    for k in [0, 3, 6]:
        for d in range(-max_d, max_d + 1):
            if d > 0:
                dp.boundary[n, k, d] = 1
            elif d == 0:
                dp.boundary[n, k, d] = r
            else:
                dp.boundary[n, k, d] = 0

## Solving the stochastic dynamic program

In [ ]:
# Solve the stochastic dynamic program
value, policy = dp.solve()

## Interpreting output from the stochastic dynamic program

* What is the probability that Team A wins after scoring a touchdown in the first possession?

In [ ]:
value[0, ('A', 6, 6)]

* What should Team A do after scoring a touchdown in the first possession?

In [ ]:
policy[0, ('A', 6, 6)]

* Suppose Team A just scored a touchdown, making it 4 points ahead. How does (1) the probability of Team A winning and (2) Team A's optimal strategy change depending on which possession this happened? Why do the trends you identified make sense?

_Hint._ Write a `for` loop that prints out the information you want.

In [ ]:
d = 4
for t in range(number_of_stages):
        if t == number_of_stages - 1:
            this_policy = 'game over'
        else:
            this_policy = str(policy[t, ('A', 6, d)])
        print("Points ahead: {1:2}  Possession: {0:2}    Go for: {2:<9}  Pr(win): {3:.4f}".format(t, d, this_policy, value[t, ('A', 6, d)]))